This notebook demonstrates how to use OptKeras, a Python wrapper around Keras and Optuna

##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    # Specify a directory in Google Drive (Modify as in your Google Drive)
    dir = '/content/drive/My Drive/Colab Notebooks/OptKeras'
    os.chdir(dir)    
    # Check the environment info
    print('## Current working directory: ', os.getcwd())
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
## Current working directory:  /content/drive/My Drive/Colab Notebooks/OptKeras
## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.0734528 hours (264.43 sec)
## Check the GPU info
Tue Feb 19 06:04:52 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------

## Install Optuna 0.7.0

In [2]:
!pip install optuna==0.7.0

    100% |████████████████████████████████| 61kB 3.0MB/s 
    100% |████████████████████████████████| 81kB 7.5MB/s 
    100% |████████████████████████████████| 51kB 23.3MB/s 
    100% |████████████████████████████████| 92kB 7.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/97/c7/868e9cd5d6a23cfb98266f30d59131e5bd441132bdf8da18d5
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna pyperclip


## Install the latest version of OptKeras

In [3]:
!pip install git+https://github.com/Minyus/optkeras.git

  Cloning https://github.com/Minyus/optkeras.git to /tmp/pip-req-build-3hnk3qwy
  Stored in directory: /tmp/pip-ephem-wheel-cache-muzcgcpy/wheels/ad/c8/9d/67471f9fd7c5fd15961a2e7aa3d15215212d0631ea96df8b5c
Successfully built optkeras


## Import modules

In [4]:
import numpy as np
import pandas as pd

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.0-rc1
Optuna 0.7.0
OptKeras 0.0.1


## Set up Dataset

In [5]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST', 'MNIST_1000samples']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## A simple Keras model

In [7]:
model = Sequential()
model.add(Conv2D(
    filters = 32, 
    kernel_size = 3, 
    strides = 1,
    activation = 'relu', 
    input_shape = input_shape ))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer = Adam(), 
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])			  
model.fit(x_train, y_train, 
          validation_data = (x_test, y_test), shuffle = True,
          batch_size = 512, epochs = 2) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 15s 257us/step - loss: 0.4482 - acc: 0.8792 - val_loss: 0.2172 - val_acc: 0.9388
Epoch 2/2
60000/60000 [==============================] - 2s 33us/step - loss: 0.1784 - acc: 0.9494 - val_loss: 0.1329 - val_acc: 0.9639


## Optimization of a simple Keras model without pruning

In [8]:
study_name = dataset_name + '_Simple'

""" Step 1. Instantiate OptKeras class
You can specify arguments for Optuna's create_study method and other arguments 
for OptKeras such as enable_pruning. 
"""

ok = OptKeras(study_name=study_name)


""" Step 2. Define objective function for Optuna """

def objective(trial):
    
    """ Clear the backend (TensorFlow). See:
    https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session
    """
    K.clear_session() 
    
    """ Step 2.1. Define parameters to try using methods of optuna.trial such as 
    suggest_categorical. In this simple demo, try 2*2*2*2 = 16 parameter sets: 
    2 values specified in list for each of 4 parameters 
    (filters, kernel_size, strides, and activation for convolution).
    """    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    """ Step 2.2. Specify callbacks(trial) and keras_verbose in fit 
    (or fit_generator) method of Keras model
    """
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    """ Step 2.3. Return trial_best_value (or latest_value) """
    return ok.trial_best_value

""" Step 3. Run optimize. 
Set n_trials and/or timeout (in sec) for optimization by Optuna
"""
ok.optimize(objective, timeout = 60) # 1 minute for demo

[2019-02-19 06:05:43.798297] Ready for optimization. (message printed as verbose is set to 1+)
[2019-02-19 06:05:44.104231] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None


[I 2019-02-19 06:05:49,396] Finished a trial resulted in value: 0.021599999332427955. Current best value is 0.021599999332427955 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}.


[2019-02-19 06:05:49.503909] Latest trial id: 0, value: 0.021599999332427955 (TrialState.COMPLETE) | Best trial id: 0, value: 0.021599999332427955, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


[I 2019-02-19 06:05:53,014] Finished a trial resulted in value: 0.07930000076293942. Current best value is 0.021599999332427955 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}.


[2019-02-19 06:05:53.125987] Latest trial id: 1, value: 0.07930000076293942 (TrialState.COMPLETE) | Best trial id: 0, value: 0.021599999332427955, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


...

[2019-02-19 06:06:33.684147] Latest trial id: 12, value: 0.03490000047683717 (TrialState.COMPLETE) | Best trial id: 0, value: 0.021599999332427955, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


[I 2019-02-19 06:06:39,169] Finished a trial resulted in value: 0.021499999427795458. Current best value is 0.021499999427795458 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}.


[2019-02-19 06:06:39.289711] Latest trial id: 13, value: 0.021499999427795458 (TrialState.COMPLETE) | Best trial id: 13, value: 0.021499999427795458, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


[I 2019-02-19 06:06:44,535] Finished a trial resulted in value: 0.020199999332427998. Current best value is 0.020199999332427998 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}.


[2019-02-19 06:06:44.567318] Latest trial id: 14, value: 0.020199999332427998 (TrialState.COMPLETE) | Best trial id: 14, value: 0.020199999332427998, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


## Randomized Grid Search of a simple Keras model

In [9]:
study_name = dataset_name + '_GridSearch'

""" To run randomized grid search, set random_grid_search_mode True """
ok = OptKeras(study_name=study_name, random_grid_search_mode=True)

def objective(trial):
   
    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    return ok.trial_best_value

""" Set the number of parameter sets as n_trials for complete grid search """
ok.random_grid_search(objective, n_trials = 2*2*2*2) # 2*2*2*2 = 16 param sets

[2019-02-19 06:06:44.597494] Ready for optimization. (message printed as verbose is set to 1+)
[2019-02-19 06:06:44.719390] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None


[I 2019-02-19 06:06:46,879] Finished a trial resulted in value: 0.06920000038146967. Current best value is 0.06920000038146967 with parameters: {'filters': 32, 'kernel_size': 3, 'strides': 2, 'activation': 'relu'}.


[2019-02-19 06:06:46.890134] Completed:   6% (    1 /    16)
[2019-02-19 06:06:46.988528] Latest trial id: 0, value: 0.06920000038146967 (TrialState.COMPLETE) | Best trial id: 0, value: 0.06920000038146967, parameters: {'filters': 32, 'kernel_size': 3, 'strides': 2, 'activation': 'relu'}


[I 2019-02-19 06:06:48,998] Finished a trial resulted in value: 0.08200000076293945. Current best value is 0.06920000038146967 with parameters: {'filters': 32, 'kernel_size': 3, 'strides': 2, 'activation': 'relu'}.


[2019-02-19 06:06:49.007544] Completed:  12% (    2 /    16)
[2019-02-19 06:06:49.110228] Latest trial id: 1, value: 0.08200000076293945 (TrialState.COMPLETE) | Best trial id: 0, value: 0.06920000038146967, parameters: {'filters': 32, 'kernel_size': 3, 'strides': 2, 'activation': 'relu'}


...

[2019-02-19 06:08:00.565609] Completed:  88% (   14 /    16)
[2019-02-19 06:08:00.684584] Latest trial id: 27, value: 0.040600000572204586 (TrialState.COMPLETE) | Best trial id: 17, value: 0.021799999332427933, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


[I 2019-02-19 06:08:03,555] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:05,682] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:07,127] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:10,697] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:12,108] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:13,509] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:16,220] Finished a trial resulted in value: 0.08239999942779541. Current best value is 0.021799999332427933 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}.


[2019-02-19 06:08:16.234885] Completed:  94% (   15 /    16)
[2019-02-19 06:08:16.352870] Latest trial id: 34, value: 0.08239999942779541 (TrialState.COMPLETE) | Best trial id: 17, value: 0.021799999332427933, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


[I 2019-02-19 06:08:18,113] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:21,120] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:22,827] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:24,444] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:27,420] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:31,014] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:32,448] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:33,874] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:36,118] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.
[I 2019-02-19 06:08:37,783] Setting trial status as TrialState.PRUNED. Trial was pruned at epoch 0.


[2019-02-19 06:08:54.000967] Completed: 100% (   16 /    16)
[2019-02-19 06:08:54.041659] Latest trial id: 52, value: 0.07879999933242798 (TrialState.COMPLETE) | Best trial id: 17, value: 0.021799999332427933, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}


## Optimization of a Keras model using more Optuna's features such as pruning

In [10]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=optuna.samplers.TPESampler(
        consider_prior=True, prior_weight=1.0, 
        consider_magic_clip=True, consider_endpoints=False, 
        n_startup_trials=10, n_ei_candidates=24, 
        seed=None), 
    pruner=optuna.pruners.SuccessiveHalvingPruner(
        min_resource=1, reduction_factor=4, min_early_stopping_rate=0), 
    study_name = study_name, 
    direction='minimize', 
    load_if_exists = True,
    # parameters for OptKeras
    monitor='val_error', # Either 'val_error' (1 - val_acc) or 'val_loss'
    enable_pruning=True, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1 )

# (Optional) Disable messages from Optuna below WARN level.
optuna.logging.set_verbosity(optuna.logging.WARN) 

def objective(trial): 
    epochs = 10
    
    K.clear_session()   
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters = int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape = input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    #batch_size = int(trial.suggest_discrete_uniform(
    #                  'Batch_size', 256, 512, 256) )
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size = batch_size,
                  epochs = epochs, validation_data = (x_test, y_test),
                  shuffle = True,
                  callbacks = ok.callbacks(trial), 
                  verbose = ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks = ok.callbacks(trial), 
                            verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, timeout = 10*60) # 10 minutes for demo

[I 2019-02-19 06:08:54,342] A new study created with name: MNIST_Optimized


[2019-02-19 06:08:54.367702] Ready for optimization. (message printed as verbose is set to 1+)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-02-19 06:08:55.388551] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-02-19 06:11:56.579468] Latest trial id: 1, value: 0.009600000000000053 (TrialState.COMPLETE) | Best trial id: 1, value: 0.009600000000000053, parameters: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.5, 'Conv_1_kernel_size': 2, 'Conv_1_max_pooling_size': 3, 'Conv_1_num_filters': 32.0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 512.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}
[2019-02-19 06:18:02.400595] Latest trial id: 11, value: 0.009099999999999997 (TrialStat

## Check the results

In [11]:
# OptKeras best_trial returns 
print('Best trial id: ', ok.best_trial.trial_id)
print('Best value:', ok.best_trial.value)
print('Best params: ')
ok.best_trial.params

Best trial id:  11
Best value: 0.009099999999999997
Best params: 


{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.0,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters': 32.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.0,
 'FC_1_num_hidden_units': 512.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 0,
 'FC_2_dropout_rate': 0.5,
 'FC_2_num_hidden_units': 512.0,
 'Optimizer': 'Adam'}

In [12]:
"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
study = ok.study
study.best_trial.params 

{'Batch_size': 256,
 'Conv': 1,
 'Conv_1_dropout_rate': 0.0,
 'Conv_1_kernel_size': 3,
 'Conv_1_max_pooling_size': 2,
 'Conv_1_num_filters': 32.0,
 'Data_augmentation': 1,
 'FC_1_acivation': 'relu',
 'FC_1_batch_normalization': 0,
 'FC_1_dropout_rate': 0.0,
 'FC_1_num_hidden_units': 512.0,
 'FC_2_acivation': 'relu',
 'FC_2_batch_normalization': 0,
 'FC_2_dropout_rate': 0.5,
 'FC_2_num_hidden_units': 512.0,
 'Optimizer': 'Adam'}

In [13]:
## Check the Optuna CSV log file 
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
pd.read_csv(ok.optuna_log_file_path)

Data Frame read from MNIST_Optimized_Optuna.csv 



,trial_id,state,value,datetime_start,datetime_complete,Batch_size,Conv,Conv_1_dropout_rate,Conv_1_kernel_size,Conv_1_max_pooling_size,...,intermediate_values_0,intermediate_values_1,intermediate_values_2,intermediate_values_3,intermediate_values_4,intermediate_values_5,intermediate_values_6,intermediate_values_7,intermediate_values_8,intermediate_values_9
0,1,TrialState.COMPLETE,0.0096,2019-02-19 06:08:54.383733,2019-02-19 06:11:55.435378,256,1,0.5,2.0,3.0,...,0.0240,0.0171,0.0118,0.0134,0.0116,0.0105,0.0102,0.0106,0.0100,0.0096
1,2,TrialState.PRUNED,0.0315,2019-02-19 06:11:55.471142,2019-02-19 06:12:32.584937,256,0,NaN,NaN,NaN,...,0.0445,0.0315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,TrialState.PRUNED,0.0173,2019-02-19 06:12:32.604743,2019-02-19 06:12:42.537040,256,1,0.5,3.0,2.0,...,0.0183,0.0173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,TrialState.PRUNED,0.0214,2019-02-19 06:12:42.557278,2019-02-19 06:13:18.519587,256,0,NaN,NaN,NaN,...,0.0360,0.0214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,10,TrialState.PRUNED,0.0300,2019-02-19 06:14:56.949029,2019-02-19 06:15:03.633743,256,0,NaN,NaN,NaN,...,0.0386,0.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11,TrialState.COMPLETE,0.0091,2019-02-19 06:15:03.662993,2019-02-19 06:18:01.183166,256,1,0.0,3.0,2.0,...,0.0208,0.0151,0.0136,0.0117,0.0115,0.0083,0.0093,0.0107,0.0104,0.0091
11,12,TrialState.PRUNED,0.0183,2019-02-19 06:18:01.236386,2019-02-19 06:18:38.937226,256,1,0.0,2.0,3.0,...,0.0204,0.0183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13,TrialState.PRUNED,0.0346,2019-02-19 06:18:38.962716,2019-02-19 06:19:16.441947,256,0,NaN,NaN,NaN,...,0.0497,0.0346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
## Check the Keras CSV log file

pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
pd.read_csv(ok.keras_log_file_path)

Data Frame read from MNIST_Optimized_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,0,2019-02-19 06:08:56.119901,2019-02-19 06:09:14.208986,val_error,1,0.860026,0.139974,0.432818,0.9760,0.0240,0.071521
1,1,2019-02-19 06:09:14.927605,2019-02-19 06:09:31.865061,val_error,1,0.941969,0.058031,0.181730,0.9829,0.0171,0.049296
2,2,2019-02-19 06:09:32.756224,2019-02-19 06:09:49.897006,val_error,1,0.954255,0.045745,0.147522,0.9882,0.0118,0.038795
3,3,2019-02-19 06:09:50.878222,2019-02-19 06:10:08.118410,val_error,1,0.960029,0.039971,0.125914,0.9866,0.0134,0.040229
...,...,...,...,...,...,...,...,...,...,...,...
38,0,2019-02-19 06:18:03.330970,2019-02-19 06:18:20.798756,val_error,12,0.914730,0.085270,0.275146,0.9796,0.0204,0.061174
39,1,2019-02-19 06:18:21.656352,2019-02-19 06:18:37.934356,val_error,12,0.962339,0.037661,0.121282,0.9817,0.0183,0.052814
40,0,2019-02-19 06:18:41.093189,2019-02-19 06:18:58.527986,val_error,13,0.813835,0.186165,0.587297,0.9503,0.0497,0.166245
41,1,2019-02-19 06:18:59.483394,2019-02-19 06:19:15.497729,val_error,13,0.907756,0.092244,0.298405,0.9654,0.0346,0.114381
